# Function Calling

In [19]:
from openai import OpenAI
import json
from dotenv import dotenv_values

client:OpenAI = OpenAI(api_key=dotenv_values(".env").get("OPENAI_API_KEY"))
GPT_MODEL = "gpt-3.5-turbo-1106"

In [20]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location:str, unit:str = "fahrenheit")->str:
    """Get the current weather in a given location"""
    if "tokyo" == location.lower():
        return json.dumps({"location":"Tokyo", "temperature":"50", "unit":"fahrenheit"})
    elif "san francisco" == location.lower():
        return json.dumps({"location":"san francisco","temperature":"40", "unit":"fahrenheit" })
    else:
        return json.dumps({"location":"Pakistan","temperature":"40", "unit":"fahrenheit" })


In [95]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
from typing import Union

def run_conversation(main_request:str)->str:
    messages:list[dict[str, str]] = [{"role":"user", "content":main_request}]
    tools:list[dict[str, Union[str, dict]]] = [
        {
            "type":"function",
            "function":{
                "name":"get_current_weather",
                "description":"Get the current weather in a given location",
                "parameters":{
                    "type":"object",
                    "properties":{
                        "location":{
                            "type":"string",
                            "description":"The city and state, e.g. San Francisco, CA"
                        },
                        "unit":{"type":"string","enum":["celsius","fahrenheit"]}
                    },
                    "required":["location"]
                }
            }
        }
    ]

    response:ChatCompletion = client.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    response_message:ChatCompletionMessage = response.choices[0].message
    display("* First Response: ",dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ",tool_calls)

    if tool_calls:
         available_functions={
              "get_current_weather":get_current_weather
         }

         messages.append(response_message)

         for tool_call in tool_calls:
             function_name = tool_call.function.name
             function_to_call = available_functions[function_name]
             function_args =json.loads(tool_call.function.arguments)
             function_response = function_to_call(
                 location = function_args.get("location"),
                 unit =  function_args.get("unit")
             )
             messages.append({
                 "tool_call_id":tool_call.id,
                 "role": "tool",
                 "name":function_name,
                 "content":function_response
             })
       
         second_response:ChatCompletion = client.chat.completions.create(
             model=GPT_MODEL,
             messages=messages,
         )

         display("* Second Response: ", dict(second_response))
         return second_response.choices[0].message.content



             
              


run_conversation("What's the weather like in Pakistan?")


'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_LVxE59SRQKkSWdrzNfnla8M0', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_LVxE59SRQKkSWdrzNfnla8M0', function=Function(arguments='{"location":"Pakistan","unit":"celsius"}', name='get_current_weather'), type='function')]

'* Second Response: '

{'id': 'chatcmpl-8SXJz7INKFhjr2TNE6WqKtaRGcxqO',
 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).', role='assistant', function_call=None, tool_calls=None))],
 'created': 1701811963,
 'model': 'gpt-3.5-turbo-1106',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_eeff13170a',
 'usage': CompletionUsage(completion_tokens=15, prompt_tokens=63, total_tokens=78)}

'The current weather in Pakistan is 40 degrees Celsius (104 degrees Fahrenheit).'

In [88]:
# from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# # from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

# def run_conversation(main_request: str)->str:
#     # Step 1: send the conversation and available functions to the model
#     messages = [{"role": "user", "content": main_request}]
#     tools = [
#         {
#             "type": "function",
#             "function": {
#                 "name": "get_current_weather",
#                 "description": "Get the current weather in a given location",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "location": {
#                             "type": "string",
#                             "description": "The city and state, e.g. San Francisco, CA",
#                         },
#                         "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
#                     },
#                     "required": ["location"],
#                 },
#             },
#         }
#     ]

#     # First Request
#     response: ChatCompletion = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",
#         messages=messages,
#         tools=tools,
#         tool_choice="auto",  # auto is default, but we'll be explicit
#     )
#     response_message: ChatCompletionMessage = response.choices[0].message
#     display("* First Response: ", dict(response_message))

#     tool_calls = response_message.tool_calls
#     display("* First Reponse Tool Calls: ", list(tool_calls))

#     # Step 2: check if the model wanted to call a function
#     if tool_calls:
#         # Step 3: call the function
#         # Note: the JSON response may not always be valid; be sure to handle errors
#         available_functions = {
#             "get_current_weather": get_current_weather,
#         }  # only one function in this example, but you can have multiple
        
#         messages.append(response_message)  # extend conversation with assistant's reply
        
#         # Step 4: send the info for each function call and function response to the model
#         for tool_call in tool_calls:
#             function_name = tool_call.function.name
#             function_to_call = available_functions[function_name]
#             function_args = json.loads(tool_call.function.arguments)
#             function_response = function_to_call(
#                 location=function_args.get("location"),
#                 unit=function_args.get("unit"),
#             )
#             messages.append(
#                 {
#                     "tool_call_id": tool_call.id,
#                     "role": "tool",
#                     "name": function_name,
#                     "content": function_response,
#                 }
#             )  # extend conversation with function response
#         display("* Second Request Messages: ", list(messages))
#         second_response: ChatCompletion = client.chat.completions.create(
#             model="gpt-3.5-turbo-1106",
#             messages=messages,
#         )  # get a new response from the model where it can see the function response
#         print("* Second Response: ", dict(second_response))
#         return second_response.choices[0].message.content
# run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_RYBS7jBRZdvYISxt1YcIKds6', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_VB8mE2IV0Rn3TVxjzrXoAQFv', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_wZG7cf0SlRPrxrwFYDnJFnDU', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_RYBS7jBRZdvYISxt1YcIKds6', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_VB8mE2IV0Rn3TVxjzrXoAQFv', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_wZG7cf0SlRPrxrwFYDnJFnDU', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_RYBS7jBRZdvYISxt1YcIKds6', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_VB8mE2IV0Rn3TVxjzrXoAQFv', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_wZG7cf0SlRPrxrwFYDnJFnDU', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_RYBS7jBRZdvYISxt1YcIKds6',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "san francisco", "temperature": "40", "unit": "fahrenheit"}'},
 {'tool_call_id': 'call_VB8mE2IV0Rn3TVxjzrXoAQFv',
  'role': 

* Second Response:  {'id': 'chatcmpl-8SXBApIpzdBUrgnMRaaYAppIsHP6X', 'choices': [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Currently, in San Francisco, the temperature is 40°F. In Tokyo, the temperature is 50°F. And in Paris, the temperature is 40°F.', role='assistant', function_call=None, tool_calls=None))], 'created': 1701811416, 'model': 'gpt-3.5-turbo-1106', 'object': 'chat.completion', 'system_fingerprint': 'fp_eeff13170a', 'usage': CompletionUsage(completion_tokens=34, prompt_tokens=170, total_tokens=204)}


'Currently, in San Francisco, the temperature is 40°F. In Tokyo, the temperature is 50°F. And in Paris, the temperature is 40°F.'